<a href="https://colab.research.google.com/github/UtkarshAIITB/IPL-Data-Analysis/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/new.csv')
# new.head()

In [4]:
t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/t.csv')
# t.head()

In [5]:
t.drop(columns = ['Unnamed: 0'], inplace = True)
t.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,597998,2013-04-03,9,6,9,9,1,0.1,6,9,2013,1,0,0,128,0,1
1,597998,2013-04-03,9,6,9,9,1,0.2,6,9,2013,1,0,0,128,4,1
2,597998,2013-04-03,9,6,9,9,1,0.3,6,9,2013,1,0,0,128,5,1
3,597998,2013-04-03,9,6,9,9,1,0.4,6,9,2013,1,0,0,128,5,1
4,597998,2013-04-03,9,6,9,9,1,0.5,6,9,2013,1,0,0,128,5,1


In [6]:
new.drop(columns = ['Unnamed: 0'], inplace = True)

In [7]:
new.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,9,5,2008,1,0,0,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,9,5,2008,1,0,0,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,9,5,2008,1,0,0,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,9,5,2008,1,0,0,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,9,5,2008,1,0,0,222,2,0


In [8]:
# new is the training set and t is the test set

In [9]:
tnew = t.drop(t[(t.over <= 11.0) | (t.over >=16.0)].index)
tnew.reset_index(drop = True)
y1test = tnew['total_runs_y']
y2test = tnew['winner']
tnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
tnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,9,6,9,1,11.1,6,9,1,0,0,76,3
1,9,6,9,1,11.2,6,9,1,0,0,77,3
2,9,6,9,1,11.3,6,9,1,0,0,78,3
3,9,6,9,1,11.4,6,9,1,0,0,79,3
4,9,6,9,1,11.5,6,9,1,0,0,79,3


In [10]:
xnew = new.drop(new[(new.over <=11.0) | (new.over >=16.0)].index)
xnew.reset_index(drop = True)
y1train = xnew['total_runs_y']
y2train = xnew['winner']
xnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
xnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,5,9,5,1,11.1,9,5,1,0,0,102,1
1,5,9,5,1,11.2,9,5,1,0,0,108,1
2,5,9,5,1,11.3,9,5,1,0,0,110,1
3,5,9,5,1,11.4,9,5,1,0,0,111,1
4,5,9,5,1,11.5,9,5,1,0,0,111,1


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [12]:
xnewt = sc.fit_transform(xnew) 
tnewt = sc.fit_transform(tnew)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [14]:
model = Sequential()
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(100, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(50, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [15]:
model.fit(x=xnewt, y=y1train, epochs=140, 
          validation_data=(tnewt,y1test))

Epoch 1/140
1370/1370 [==============================] - 4s 2ms/step - loss: 2790.8967 - val_loss: 328.8148
Epoch 2/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1590.1355 - val_loss: 282.4111
Epoch 3/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1527.5303 - val_loss: 343.6303
Epoch 4/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1449.0907 - val_loss: 264.9811
Epoch 5/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1399.4480 - val_loss: 256.8339
Epoch 6/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1368.4690 - val_loss: 295.4435
Epoch 7/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1299.6635 - val_loss: 244.6853
Epoch 8/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1275.7572 - val_loss: 256.4102
Epoch 9/140
1370/1370 [==============================] - 3s 2ms/step - loss: 1247.0596 - val_loss: 259.3876
Epoch 10/140
1370/1370 [====

In [16]:
y1test = y1test.ravel()

In [17]:
predictions = model.predict(tnewt)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=y1test
sample.head(10)

,Predict,Actual
0,163.520981,128
1,164.069519,128
2,164.637711,128
3,165.199371,128
4,164.716629,128
5,145.522430,128
6,145.779633,128
7,145.437988,128
8,144.952209,128
9,149.125854,128


In [18]:
def custom_accuracy(y_test,y_pred,thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right += 1
    return ((right/l)*100)

In [19]:
custom_accuracy(sample['Actual'] , sample['Predict'],10)

42.28583916083916

In [20]:
# sample.to_csv('sample.csv')

In [21]:
# try1 = sample[sample.Actual == 128]
# try1.head()

In [22]:
# try1.shape[0]

In [23]:
# try1.sum()

In [24]:
# (try1.sum())/try1.shape[0]

In [25]:
# try2 = sample[sample.Actual == 129]
# (try2.sum())/try2.shape[0]

In [26]:
# try2 = sample[sample.Actual == 156]
# result = (try2.sum())/try2.shape[0]

In [27]:
# result

In [28]:
# res2 = result.values.reshape(1,2)

In [29]:
# res2

In [30]:
# res2.shape

In [31]:
# df.loc[len(df)] = your_array
# results.loc[len(results)] = res2
# df.append(pd.DataFrame(arr).T)
# results = results.append(pd.DataFrame(res2))
# results.append(pd.DataFrame(res3))

In [32]:
ar1 = sample['Actual'].unique()

In [33]:
results = pd.DataFrame()

In [34]:
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [35]:
results.head()

,0,1
0,145.289126,128.0
0,149.899839,129.0
0,150.778961,156.0
0,159.238173,154.0
0,144.660579,126.0


In [36]:
# rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)

In [37]:
results.reset_index(drop = True)

,Predict,Actual
0,145.289126,128.0
1,149.899839,129.0
2,150.778961,156.0
3,159.238173,154.0
4,144.660579,126.0
...,...,...
79,196.608810,190.0
80,126.905037,113.0
81,161.725775,172.0
82,146.324529,134.0


In [38]:
results.tail()

,Predict,Actual
0,196.608810,190.0
0,126.905037,113.0
0,161.725775,172.0
0,146.324529,134.0
0,169.940186,192.0


In [39]:
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

53.57142857142857


In [40]:
# results.info()

In [41]:
# results['Actual']

take epochs = 90 next time

In [42]:
tests = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/testset.csv')
# tests.head()
tests.drop(columns = ['Unnamed: 0'], inplace = True)
tests.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.1,12.0,5.0,1,0,0,159,2,0
1,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.2,12.0,5.0,1,0,0,159,2,0
2,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.3,12.0,5.0,1,0,0,159,2,0
3,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.4,12.0,5.0,1,0,0,159,4,0
4,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.5,12.0,5.0,1,0,0,159,4,0


In [43]:
sets = tests.drop(tests[(tests.over <= 11.0) | (tests.over >=16.0)].index)
sets.reset_index(drop = True)
yt1 = sets['total_runs_y']
yt2 = sets['winner']
sets.drop(columns = ['id', 'date', 'total_runs_y', 'winner'], inplace = True)
sets.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,12.0,5.0,5.0,1,11.1,12.0,5.0,1,0,0,95,2
1,12.0,5.0,5.0,1,11.2,12.0,5.0,1,0,0,95,2
2,12.0,5.0,5.0,1,11.3,12.0,5.0,1,0,0,96,2
3,12.0,5.0,5.0,1,11.4,12.0,5.0,1,0,0,96,2
4,12.0,5.0,5.0,1,11.5,12.0,5.0,1,0,0,97,2


In [44]:
yt1 = yt1.ravel()

In [45]:
xs = sc.fit_transform(sets)

In [46]:
predictions = model.predict(xs)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=yt1
sample.head(10)

,Predict,Actual
0,167.983765,159
1,167.366104,159
2,167.485123,159
3,166.782379,159
4,166.883209,159
5,166.991333,159
6,164.669586,159
7,166.712204,159
8,166.728973,159
9,166.707779,159


In [47]:
ar1 = sample['Actual'].unique()
results = pd.DataFrame()
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [52]:
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)
results.reset_index(drop = True)
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

37.142857142857146
